In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [2]:
import zipfile
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!zip -r "/content/drive/MyDrive/dataset.zip" "/content/drive/MyDrive/dataset"

updating: content/drive/MyDrive/dataset/ (stored 0%)
updating: content/drive/MyDrive/dataset/train/ (stored 0%)
updating: content/drive/MyDrive/dataset/train/class3/ (stored 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (77).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (7).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (80).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (75).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (70).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (8).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (73).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (76).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (69).jpg (deflated 0%)
updating: content/drive/MyDrive/dataset/train/class3/Dust (72).jpg (deflated 0%)
updating: cont

In [5]:
# Path to your .zip file in Google Drive
zip_path = "/content/drive/MyDrive/dataset.zip"

# Folder to extract to
extract_path = "/content/dataset"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped to:", extract_path)

✅ Unzipped to: /content/dataset


In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
data_dir = '/content/dataset/content/drive/MyDrive/dataset'

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#image_datasets

In [8]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

{'train': 634, 'val': 235}


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


['class1', 'class2', 'class3', 'class4', 'class5', 'class6']

In [9]:
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer (assuming 6 classes)
num_classes = 6
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Unfreeze the last block (layer4) and the final fc layer
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define loss function
criterion = nn.CrossEntropyLoss()

# Use Adam optimizer with a smaller learning rate
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# Optional: Learning rate scheduler (decays LR every 5 epochs)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]


In [10]:
import copy
import time

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
since = time.time()

num_epochs = 10
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print('-' * 30)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')

        # Deep copy best model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), 'best_model.pth')
            print("✅ Best model updated and saved.")

time_elapsed = time.time() - since
print(f"\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
print(f"Best Validation Accuracy: {best_acc:.4f}")

# Load best weights
model.load_state_dict(best_model_wts)



Epoch 1/10
------------------------------
Train Loss: 1.1473 | Acc: 0.5710
Val Loss: 0.6905 | Acc: 0.7489
✅ Best model updated and saved.

Epoch 2/10
------------------------------
Train Loss: 0.8701 | Acc: 0.6924
Val Loss: 0.5562 | Acc: 0.8085
✅ Best model updated and saved.

Epoch 3/10
------------------------------
Train Loss: 0.7948 | Acc: 0.7192
Val Loss: 0.5292 | Acc: 0.8426
✅ Best model updated and saved.

Epoch 4/10
------------------------------
Train Loss: 0.7064 | Acc: 0.7603
Val Loss: 0.5260 | Acc: 0.8638
✅ Best model updated and saved.

Epoch 5/10
------------------------------
Train Loss: 0.6596 | Acc: 0.7618
Val Loss: 0.6298 | Acc: 0.8128

Epoch 6/10
------------------------------
Train Loss: 0.6044 | Acc: 0.7792
Val Loss: 0.5586 | Acc: 0.8553

Epoch 7/10
------------------------------
Train Loss: 0.5222 | Acc: 0.8060
Val Loss: 0.5287 | Acc: 0.8638

Epoch 8/10
------------------------------
Train Loss: 0.5476 | Acc: 0.8218
Val Loss: 0.5290 | Acc: 0.8468

Epoch 9/10
----

<All keys matched successfully>

In [11]:
# Save the model
torch.save(model.state_dict(), 'solarr_classification_model.pth')